In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
pd.set_option('display.max.columns', 100)
# to draw pictures in jupyter notebook
%matplotlib inline 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
user_usage = pd.read_csv('../../ML_datasets/user_usage.csv')
user_usage.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id
0,21.97,4.82,1557.33,22787
1,1710.08,136.88,7267.55,22788
2,1710.08,136.88,7267.55,22789
3,94.46,35.17,519.12,22790
4,71.59,79.26,1557.33,22792


In [3]:
user_device = pd.read_csv('../../ML_datasets/user_device.csv')
user_device.head()

,use_id,user_id,platform,platform_version,device,use_type_id
0,22782,26980,ios,10.2,"iPhone7,2",2
1,22783,29628,android,6.0,Nexus 5,3
2,22784,28473,android,5.1,SM-G903F,1
3,22785,15200,ios,10.2,"iPhone7,2",3
4,22786,28239,android,6.0,ONE E1003,1


In [4]:
%%time
join1 = pd.merge(user_usage, user_device,on='use_id')

Wall time: 16 ms


In [5]:
join1.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,user_id,platform,platform_version,device,use_type_id
0,21.97,4.82,1557.33,22787,12921,android,4.3,GT-I9505,1
1,1710.08,136.88,7267.55,22788,28714,android,6.0,SM-G930F,1
2,1710.08,136.88,7267.55,22789,28714,android,6.0,SM-G930F,1
3,94.46,35.17,519.12,22790,29592,android,5.1,D2303,1
4,71.59,79.26,1557.33,22792,28217,android,5.1,SM-G361F,1


In [8]:
%%time
query1 = '''
    SELECT *
    FROM user_usage A
    JOIN user_device B
    ON A.use_id = B.use_id
    '''
join2 = ps.sqldf(query1, locals())

Wall time: 44 ms


In [9]:
join2.head()

,outgoing_mins_per_month,outgoing_sms_per_month,monthly_mb,use_id,use_id,user_id,platform,platform_version,device,use_type_id
0,21.97,4.82,1557.33,22787,22787,12921,android,4.3,GT-I9505,1
1,1710.08,136.88,7267.55,22788,22788,28714,android,6.0,SM-G930F,1
2,1710.08,136.88,7267.55,22789,22789,28714,android,6.0,SM-G930F,1
3,94.46,35.17,519.12,22790,22790,29592,android,5.1,D2303,1
4,71.59,79.26,1557.33,22792,22792,28217,android,5.1,SM-G361F,1


In [10]:
%%time
aggr_query = '''
    SELECT
        avg(outgoing_sms_per_month) as avg_sms_per_month,
        device
    FROM user_usage A
    JOIN user_device B
    ON A.use_id = B.use_id
    GROUP BY device
    '''
aggr_join1 = ps.sqldf(aggr_query, locals())

Wall time: 40 ms


In [11]:
aggr_join1.head()

,avg_sms_per_month,device
0,62.100,A0001
1,162.390,C6603
2,35.375,D2303
3,48.670,D5503
4,105.950,D5803


In [17]:
%%time
aggregation = {
    'outgoing_sms_per_month':'mean'
}
aggr_join2 = pd.merge(user_usage[['use_id','outgoing_sms_per_month']],
                      user_device[['use_id','device']],on='use_id').groupby('device').agg(aggregation)

Wall time: 24 ms


In [18]:
aggr_join2.head()

,outgoing_sms_per_month
device,
A0001,62.100
C6603,162.390
D2303,35.375
D5503,48.670
D5803,105.950
